# Lab #1 (Part 5): Ames 주택 가격 예측 모델 성능 종합 평가

지금까지 배운 4가지 회귀 평가지표를 모두 사용하여, 실제 데이터셋에 대한 모델 성능을 종합적으로 분석하는 실습을 진행해 보겠습니다. 

**Ames Housing 데이터셋**을 사용하여 두 개의 다른 회귀 모델(`LinearRegression`, `RandomForestRegressor`)을 만들고, 어떤 모델이 어떤 지표에서 더 우수한지 비교 분석합니다.

### 과제 목표
1. `Ames Housing` 데이터를 로드하고 기본적인 전처리를 수행합니다.
   
2. `LinearRegression`과 `RandomForestRegressor` 모델을 훈련시킵니다.
3. 두 모델의 성능을 **MAE, RMSE, R², RMSLE** 네 가지 지표로 각각 평가합니다.
4. 평가 결과를 DataFrame으로 정리하고, 시각화를 통해 모델의 예측 경향을 분석합니다.

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
import plotly.express as px

# ---------------------------------------------------
# 1. 데이터 로드 및 준비 (Data Loading and Preparation)
# ---------------------------------------------------
# Ames Housing 데이터셋 로드
housing = fetch_openml(name="house_prices", as_frame=True)
df = housing.data.join(housing.target)
df.rename(columns={'target': 'SalePrice'}, inplace=True)

# 특성(X)과 타겟(y) 분리
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# 실습의 편의를 위해 수치형 특성만 선택
X_numeric = X.select_dtypes(include=np.number)

# 결측치를 각 열의 평균값으로 채우기
X_numeric = X_numeric.fillna(X_numeric.mean())

# 훈련 데이터와 테스트 데이터 분리 (8:2 비율, 랜덤 시드 고정)
X_train, X_test, y_train, y_test = train_test_split(
    X_numeric, y, test_size=0.2, random_state=42
)

# ---------------------------------------------------
# 2. 모델 훈련 (Model Training)
# ---------------------------------------------------
# [문제 1] LinearRegression 모델을 생성하고 훈련시키세요.
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# [문제 2] RandomForestRegressor 모델을 생성하고 훈련시키세요.
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# ---------------------------------------------------
# 3. 예측 및 평가 (Prediction and Evaluation)
# ---------------------------------------------------
# 테스트 데이터에 대한 예측 수행
lr_preds = lr_model.predict(X_test)
rf_preds = rf_model.predict(X_test)

# 각 모델에 대한 평가지표 계산 함수
def evaluate_regression(y_true, y_pred, model_name=""):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    # RMSLE 계산 시 음수 예측값이 있을 경우를 대비하여 0으로 클리핑
    y_pred_clipped = np.clip(y_pred, 0, None)
    rmsle = np.sqrt(mean_squared_log_error(y_true, y_pred_clipped))

    print(f"--- {model_name} 모델 평가 결과 ---")
    print(f"MAE: {mae:,.2f}")
    print(f"RMSE: {rmse:,.2f}")
    print(f"R²: {r2:.4f}")
    print(f"RMSLE: {rmsle:.4f}\n")
    return [mae, rmse, r2, rmsle]

# [문제 3] LinearRegression 모델의 성능을 평가하세요.
lr_metrics = evaluate_regression(y_test, lr_preds, model_name="Linear Regression")

# [문제 4] RandomForestRegressor 모델의 성능을 평가하세요.
rf_metrics = evaluate_regression(y_test, rf_preds, model_name="Random Forest")

# ---------------------------------------------------
# 4. 결과 분석 및 시각화 (Result Analysis and Visualization)
# ---------------------------------------------------
# [문제 5] 평가 결과를 pandas DataFrame으로 만드세요.
metrics_data = {
    'Metric': ['MAE', 'RMSE', 'R²', 'RMSLE'],
    'Linear Regression': lr_metrics,
    'Random Forest': rf_metrics
}
metrics_df = pd.DataFrame(metrics_data).set_index('Metric')
# 소수점 자리 정리
metrics_df['Linear Regression'] = metrics_df['Linear Regression'].apply(lambda x: f"{x:,.4f}")
metrics_df['Random Forest'] = metrics_df['Random Forest'].apply(lambda x: f"{x:,.4f}")
print(metrics_df)

# [문제 6] RandomForest 모델의 예측 결과를 시각화하세요.
fig = px.scatter(x=y_test, y=rf_preds,
                 title='실제값 vs 예측값 (Random Forest)',
                 labels={'x': '실제 가격', 'y': '예측 가격'},
                 opacity=0.6)
# 완벽한 예측을 의미하는 y=x 직선(빨간색)을 추가하세요.
fig.add_shape(type='line', x0=y_test.min(), y0=y_test.min(),
              x1=y_test.max(), y1=y_test.max(), line=dict(color='Red', dash='dash'))
fig.show()